In [1]:
from src.load_data import load_data

position_info, spikes, multiunit_firing_rate, multiunit_HSE_times = load_data()


In [3]:
from replay_trajectory_classification import (
    SortedSpikesClassifier,
    Environment,
    RandomWalk,
    Uniform,
)


environment = Environment(place_bin_size=3.0)
continuous_transition_types = [
    [RandomWalk(movement_var=12.0), Uniform()],
    [Uniform(), Uniform()],
]


classifier = SortedSpikesClassifier(
    environments=environment,
    continuous_transition_types=continuous_transition_types,
    sorted_spikes_algorithm="spiking_likelihood_kde_gpu",
    sorted_spikes_algorithm_params={
        "position_std": 6.0,
        "use_diffusion": False,
        "block_size": int(2**12),
    },
)

classifier


SortedSpikesClassifier(continuous_transition_types=[[RandomWalk(environment_name='',
                                                                movement_var=12.0,
                                                                movement_mean=0.0,
                                                                use_diffusion=False),
                                                     Uniform(environment_name='',
                                                             environment2_name=None)],
                                                    [Uniform(environment_name='',
                                                             environment2_name=None),
                                                     Uniform(environment_name='',
                                                             environment2_name=None)]],
                       discrete_transition_type=DiagonalDiscrete(diagona...
                       infer_track_interior=True,
                       initial_conditions_type=UniformInitialConditions(),
                       observation_models=(ObservationModel(environment_name='',
                                                            encoding_group=0),
                                           ObservationModel(environment_name='',
                                                            encoding_group=0)),
                       sorted_spikes_algorithm='spiking_likelihood_kde_gpu',
                       sorted_spikes_algorithm_params={'block_size': 4096,
                                                       'position_std': 6.0,
                                                       'use_diffusion': False})

In [ ]:
import cupy as cp


state_names = ["continuous", "fragmented"]

GPU_ID = 5

# use context manager to specify which GPU (device)
with cp.cuda.Device(GPU_ID):
    # Fit the model place fields
    classifier.fit(position=position_info[["x", "y"]], spikes=spikes)

    # Run the model on the simulated replay
    results = classifier.predict(
        spikes,
        time=spikes.index,
        state_names=state_names,
        use_gpu=True,
    )
